<a href="https://colab.research.google.com/github/graphtrek/stockforecast/blob/main/tasty_annual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, time
import numpy as np
import math
import time
import json

from pathlib import Path

import pandas as pd

In [18]:
file_path = '/content/drive/MyDrive/tastyworks_transactions_x5706_2021-01-01_2021-12-31.csv'
file_exists = os.path.exists(file_path)
if file_exists is True:
  tasty_df = pd.read_csv(file_path)
  tasty_df = tasty_df.reset_index()  # make sure indexes pair with number of rows
  print("Tasty History loaded:", file_path)
else:
  print("Tasty History does not exists:", file_path)


Tasty History loaded: /content/drive/MyDrive/tastyworks_transactions_x5706_2021-01-01_2021-12-31.csv


In [31]:
def find_closed_transactions(symbol,df):
  return df.query('Action.str.contains("CLOSE")  and Symbol=="' + symbol + '"', engine='python') 

In [32]:

for index, row in tasty_df.iterrows():
  if "OPEN" in str(row['Action']):
    closed_df = find_closed_transactions(str(row['Symbol']), tasty_df)
    print(closed_df['Symbol'], closed_df['Quantity'])    
    #print(row['Date'], row['Action'], row['Symbol'], row['Quantity'])

Series([], Name: Symbol, dtype: object) Series([], Name: Quantity, dtype: float64)
Series([], Name: Symbol, dtype: object) Series([], Name: Quantity, dtype: float64)
Series([], Name: Symbol, dtype: object) Series([], Name: Quantity, dtype: float64)
Series([], Name: Symbol, dtype: object) Series([], Name: Quantity, dtype: float64)
Series([], Name: Symbol, dtype: object) Series([], Name: Quantity, dtype: float64)
Series([], Name: Symbol, dtype: object) Series([], Name: Quantity, dtype: float64)
Series([], Name: Symbol, dtype: object) Series([], Name: Quantity, dtype: float64)
Series([], Name: Symbol, dtype: object) Series([], Name: Quantity, dtype: float64)
Series([], Name: Symbol, dtype: object) Series([], Name: Quantity, dtype: float64)
31    APPS  220318C00060000
Name: Symbol, dtype: object 31    1.0
Name: Quantity, dtype: float64
Series([], Name: Symbol, dtype: object) Series([], Name: Quantity, dtype: float64)
Series([], Name: Symbol, dtype: object) Series([], Name: Quantity, dtype: